# Baseline: bucket-of-chump Pre-optimized Solution

This establishes the baseline using the best publicly available pre-optimized solution.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import json

getcontext().prec = 30
scale_factor = 1

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal('0.0') * scale_factor), float(tip_y * scale_factor)),
            (float(top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
            (float(top_w / Decimal('4') * scale_factor), float(tier_1_y * scale_factor)),
            (float(mid_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
            (float(mid_w / Decimal('4') * scale_factor), float(tier_2_y * scale_factor)),
            (float(base_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal('2') * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(base_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
            (float(-(mid_w / Decimal('4')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(mid_w / Decimal('2')) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(top_w / Decimal('4')) * scale_factor), float(tier_1_y * scale_factor)),
            (float(-(top_w / Decimal('2')) * scale_factor), float(tier_1_y * scale_factor)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(str(max(bounds[2] - bounds[0], bounds[3] - bounds[1]))) / scale_factor

def score_submission(csv_path):
    df = pd.read_csv(csv_path)
    df['x'] = df['x'].astype(str).str.strip().str.lstrip('s')
    df['y'] = df['y'].astype(str).str.strip().str.lstrip('s')
    df['deg'] = df['deg'].astype(str).str.strip().str.lstrip('s')
    df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)
    
    total_score = Decimal('0')
    per_n_scores = {}
    
    for group_id, group_data in df.groupby('group_id'):
        n = int(group_id)
        tree_list = [ChristmasTree(center_x=row['x'], center_y=row['y'], angle=row['deg'])
                     for _, row in group_data.iterrows()]
        side = get_tree_list_side_length(tree_list)
        score = side ** 2 / Decimal(str(n))
        total_score += score
        per_n_scores[n] = float(score)
    
    return float(total_score), per_n_scores

print('Scoring functions defined.')

Scoring functions defined.


In [2]:
# Score the bucket-of-chump baseline solution
baseline_path = '/home/code/exploration/preoptimized/submission.csv'
baseline_score, per_n_scores = score_submission(baseline_path)

print(f'Baseline (bucket-of-chump) score: {baseline_score:.6f}')
print(f'Target score: 68.919154')
print(f'Gap to target: {baseline_score - 68.919154:.6f}')
print(f'Improvement needed: {(baseline_score - 68.919154) / baseline_score * 100:.2f}%')

Baseline (bucket-of-chump) score: 70.647327
Target score: 68.919154
Gap to target: 1.728173
Improvement needed: 2.45%


In [3]:
# Copy the baseline solution to submission folder
import shutil

# Copy to submission folder
shutil.copy(baseline_path, '/home/submission/submission.csv')
print('Copied baseline to /home/submission/submission.csv')

# Also copy to experiment folder for reference
shutil.copy(baseline_path, 'submission.csv')
print('Copied baseline to experiment folder')

Copied baseline to /home/submission/submission.csv
Copied baseline to experiment folder


In [4]:
# Save metrics
metrics = {
    'cv_score': baseline_score,
    'target': 68.919154,
    'gap': baseline_score - 68.919154,
    'source': 'bucket-of-chump pre-optimized dataset'
}

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f'Saved metrics: {metrics}')

Saved metrics: {'cv_score': 70.64732689763679, 'target': 68.919154, 'gap': 1.7281728976367816, 'source': 'bucket-of-chump pre-optimized dataset'}
